# AUTO-07P Computations

This notebook executes the auto07p calculations needed to reproduce the results presented in figures 6 and 7 of [Bramburger et al.] for the Swift-Hohenberg equation: we show that wave trains in the Swift-Hohenberg equation form a torus and predict the effect of radial terms on the ODE Hamiltonian H.

## Setup

First, we import the relevant packages we need.

In [5]:
import sys
import os
sys.path.append(os.environ["AUTO_DIR"]+'/python')
import auto

import numpy as np
import pandas as pd

## Maxwell point

We calculate the Maxwell point $\mu$ at which the wave trains satisfy $H=0$ and $S=0$, where $E=S+H$ is the PDE energy associated with the Swift-Hohenberg equation.

In [6]:
sh = auto.load('sh')
task_mp = auto.load(sh, UZR={'S':0}, STOP=['UZ1'])
solution_mp = auto.run(task_mp)

Runner configured
Runner configured
gfortran -fopenmp -O -c sh.f90 -o sh.o
gfortran -fopenmp -O sh.o -o sh.exe /usr/local/src/auto/lib/*.o
Starting sh ...

  BR    PT  TY  LAB       Mu         L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             E       
   1     6  UZ    2   2.00468E-01   8.96783E-01   1.10009E+00   8.03986E-01   3.58162E-01   6.32542E+00   1.09541E-10   1.09541E-10

 Total Time    0.156E-01
sh ... done


## Computing $S=0$ in $(\mu,H)$ space

In the next step, we calculate the curve in $(\mu,H)$ space along which $S=0$ and save the results.

In [7]:
task_s0 = auto.load(solution_mp('UZ1'), e='sh', c='sh', ICP=['Mu','Period','H','S'])
solution_s0 = auto.merge(auto.run(task_s0) + auto.run(task_s0,DS='-'))
df_s0 = pd.DataFrame({'Mu':solution_s0['Mu'], 'h':solution_s0['H']})
df_s0.to_pickle('data/df_s0.pd')

Runner configured
Starting sh ...

  BR    PT  TY  LAB       Mu         L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          H             S       
   1     6  UZ    3   2.15000E-01   8.57792E-01   1.05174E+00   7.70845E-01   3.40649E-01   6.32711E+00   2.75503E-03   1.09541E-10

 Total Time    0.156E-01
sh ... done
Starting sh ...

  BR    PT  TY  LAB       Mu         L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          H             S       
   1     7  UZ    3   1.80000E-01   9.41603E-01   1.15546E+00   8.42250E-01   3.77849E-01   6.32303E+00  -4.26341E-03   1.09541E-10

 Total Time    0.157E-01
sh ... done
Merge done


/anaconda3/envs/python_for_auto/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2699: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


## Compute torus of periodic orbits

We return to the Maxwell point and continue the periodic solutions in the Hamiltonian $H$ (initially in the direction of increasing and then decreasing values of $H$) and stop when $H$ reaches the value zero.

In [8]:
task_torus = auto.load(solution_mp('UZ1'), e='sh', c='sh', ICP=['H','Period','S','Mu'], UZR={-3:0.0}, NINT=1, NPR=4)
solution_torus = auto.merge(auto.run(task_torus) + auto.run(task_torus,DS='-'))
data = solution_torus()

x = np.array([list(s['U(1)']) for s in data])
y = np.array([list(s['U(2)']) for s in data])
z = np.array([list(s['U(5)']) for s in data])

np.save("data/torus_x",x)
np.save("data/torus_y",y)
np.save("data/torus_z",z)

Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             Mu      
   1     4        3   5.14302E-03   8.95160E-01   1.10017E+00   8.00696E-01   3.58214E-01   6.35354E+00   1.09541E-10   2.00468E-01
   1     8        4   1.17667E-02   8.92741E-01   1.09981E+00   7.95900E-01   3.58367E-01   6.39114E+00   1.09541E-10   2.00468E-01
   1    12        5   1.81123E-02   8.90036E-01   1.09892E+00   7.90753E-01   3.58546E-01   6.42886E+00   1.09541E-10   2.00468E-01
   1    16        6   2.41700E-02   8.87054E-01   1.09752E+00   7.85281E-01   3.58869E-01   6.46668E+00   1.09541E-10   2.00468E-01
   1    20        7   2.99313E-02   8.83796E-01   1.09563E+00   7.79496E-01   3.59202E-01   6.50461E+00   1.09541E-10   2.00468E-01
   1    24        8   3.53898E-02   8.80265E-01   1.09325E+00   7.73395E-01   3.59567E-01   6.54264E+00   1.09541E-10   2.00468E-01
   1    28        9   4.05400E-02   8.76

   1    20        7  -3.55862E-02   9.02718E-01   1.09146E+00   8.20184E-01   3.58805E-01   6.14881E+00   1.09541E-10   2.00468E-01
   1    24        8  -4.35962E-02   9.02823E-01   1.08759E+00   8.22196E-01   3.59139E-01   6.11195E+00   1.09541E-10   2.00468E-01
   1    28        9  -5.17139E-02   9.02422E-01   1.08285E+00   8.23580E-01   3.59492E-01   6.07518E+00   1.09541E-10   2.00468E-01
   1    32       10  -5.98936E-02   9.01437E-01   1.07717E+00   8.24248E-01   3.59838E-01   6.03851E+00   1.09541E-10   2.00468E-01
   1    36       11  -6.80792E-02   8.99770E-01   1.07041E+00   8.24106E-01   3.60145E-01   6.00192E+00   1.09541E-10   2.00468E-01
   1    40       12  -7.62003E-02   8.97290E-01   1.06243E+00   8.23040E-01   3.60377E-01   5.96542E+00   1.09541E-10   2.00468E-01
   1    44       13  -8.41666E-02   8.93821E-01   1.05303E+00   8.20886E-01   3.60401E-01   5.92899E+00   1.09541E-10   2.00468E-01
   1    48       14  -9.18589E-02   8.89124E-01   1.04195E+00   8.17423E-01 

## Computing a contour plot of S

Starting at $\mu=0.22$, we compute the periodic orbits for $H=0$ and save the orbits at regularly spaced points along the $\mu$ axis.

In [9]:
task_vf1 = auto.load(sh, UZR={}, UZSTOP={'Mu':0.22})
solution_vf1 = auto.run(task_vf1)
uzr_list = np.arange(0.181, 0.217, 0.002)
task_vf2 = auto.load(solution_vf1, UZR={'Mu':uzr_list}, UZSTOP={'Mu':0.18}, DS='-')
solution_vf2 = auto.run(task_vf2)
uz_solutions = solution_vf2("UZ")

Runner configured
Starting sh ...

  BR    PT  TY  LAB       Mu         L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             E       
   1    13  UZ    2   2.20000E-01   8.43087E-01   1.03185E+00   7.59755E-01   3.33351E-01   6.30575E+00   3.64951E-03   3.64951E-03

 Total Time    0.243E-01
sh ... done
Runner configured
Starting sh ...

  BR    PT  TY  LAB       Mu         L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             E       
   1     3  UZ    3   2.15000E-01   8.58581E-01   1.05152E+00   7.72607E-01   3.40574E-01   6.31106E+00   2.75853E-03   2.75853E-03
   1     4  UZ    4   2.13000E-01   8.64371E-01   1.05888E+00   7.77403E-01   3.43260E-01   6.31312E+00   2.39333E-03   2.39333E-03
   1     5  UZ    5   2.11000E-01   8.69964E-01   1.06598E+00   7.82027E-01   3.45848E-01   6.31515E+00   2.02331E-03   2.02331E-03
   1     6  UZ    6   2.09000E-01   8.75377E-01   1.07286E+00   7.86499E-01   3.48346E-01   6.3171

We can now compute the vector field $S$ as follows. We restart at each of the orbits saved above and continue them for fixed parameter $\mu$ in $H$ towards increasing and then decreasing values of $H$. Throughout the continuation, we compute the value of $S$ and append the values of $\mu$, $H$, and $S$ in vectors.

In [10]:
data = np.empty((3,0))
for solution in uz_solutions:
  task_vf = auto.load(solution, ICP=['H','Period','S','Mu'],STOP='LP1')
  solution_vf = auto.run(task_vf)
  data = np.hstack((data,np.vstack((solution_vf['Mu'], solution_vf['H'], solution_vf['S']))))
  task_vf = auto.load(solution, ICP=['H','Period','S','Mu'],STOP='LP1',DS='-')
  solution_vf = auto.run(task_vf)
  data = np.hstack((data,np.vstack((solution_vf['Mu'], solution_vf['H'], solution_vf['S']))))
df_surface = pd.DataFrame(np.transpose(data), columns=('Mu', 'H', 'S'))
df_surface.to_pickle('data/df_surface.pd')

Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             Mu      
   1    57  LP   22  -8.61217E-02   7.89973E-01   9.13090E-01   7.32810E-01   3.16860E-01   5.81293E+00   9.30919E-02   2.15000E-01

 Total Time    0.670E-01
sh ... done
Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             Mu      
   1    75  LP   22   6.73788E-02   7.45308E-01   9.39913E-01   6.28035E-01   3.29702E-01   7.00665E+00  -6.05994E-02   2.15000E-01

 Total Time    0.788E-01
sh ... done
Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             Mu      
   1    59  LP   22  -9.04900E-02   7.95566E-01   9.17416E-01   7.37623E-01   3.20781E-01   5.79833E+00   9.74207E-02   2.13000E-01

 Total Time    0.666E-0

   1    77  LP   22  -1.35806E-01   8.52538E-01   9.60363E-01   7.84356E-01   3.62853E-01   5.67026E+00   1.42157E-01   1.91000E-01

 Total Time    0.101E+00
sh ... done
Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             Mu      
   1    95  LP   22   8.51578E-02   7.96104E-01   1.01073E+00   6.53341E-01   3.71309E-01   7.22344E+00  -7.98470E-02   1.91000E-01

 Total Time    0.112E+00
sh ... done
Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S             Mu      
   1    78  LP   22  -1.39764E-01   8.57407E-01   9.63929E-01   7.88167E-01   3.66583E-01   5.66069E+00   1.46051E-01   1.89000E-01

 Total Time    0.104E+00
sh ... done
Runner configured
Starting sh ...

  BR    PT  TY  LAB       H          L2-NORM       MAX U(1)      MAX U(2)      MAX U(3)       Period          S        